In [17]:
import os
import pandas as pd
from pathlib import Path

def check_gold_truth_mistakes(base_path):
    """
    Check for differences between .xlsx and corresponding .ods files in subfolders.
    
    Args:
        base_path: Path to the main directory containing subfolders
    """
    base_path = Path(base_path)
    results = []
    count_files_without_error = 0
    no_ods = []
    count_files = 0
    count_files_with_error = 0
    
    # Iterate through all subfolders
    for subfolder in base_path.iterdir():
        if not subfolder.is_dir():
            continue
            
        print(f"Processing subfolder: {subfolder.name}")
        
        # Step 1: Find all .xlsx files that do not start with "aktiva_"
        xlsx_files = [f for f in subfolder.glob("*.xlsx") if not f.name.startswith("aktiva_")]
        
        for xlsx_file in xlsx_files:
            print(f"  Found xlsx file: {xlsx_file.name}")
            count_files += 1
            
            # Step 2: Check if corresponding .ods file exists
            base_name = xlsx_file.stem  # filename without extension
            ods_file = subfolder / f"aktiva_{base_name}_MYLA.ods"
            
            if ods_file.exists():
                print(f"    Found corresponding ods file: {ods_file.name}")
                
                try:
                    # Step 3: Load both dataframes
                    df_xlsx = pd.read_excel(xlsx_file)
                    df_ods = pd.read_excel(ods_file, engine='odf')  # Use odf engine for .ods files
                    
                    print(f"    Loaded xlsx: {df_xlsx.shape}, ods: {df_ods.shape}")
                    
                    # Check if required columns exist
                    required_cols = ['E1', 'E2', 'E3', 'year', 'previous_year']
                    
                    xlsx_has_cols = all(col in df_xlsx.columns for col in required_cols)
                    ods_has_cols = all(col in df_ods.columns for col in required_cols)
                    
                    if not xlsx_has_cols:
                        print(f"    Warning: xlsx file missing required columns")
                        continue
                    if not ods_has_cols:
                        print(f"    Warning: ods file missing required columns") 
                        continue
                    
                    # Step 4: Join dataframes and check for differences
                    merged = df_xlsx.merge(df_ods, on=['E1', 'E2', 'E3'], 
                                         suffixes=('_xlsx', '_ods'), how='inner')
                    
                    if merged.empty:
                        print(f"    Warning: No matching rows found for join")
                        continue
                    
                    # Check for differences in year and previous_year columns
                    year_diff = ~(
                        (merged['year_xlsx'] == merged['year_ods']) |
                        (merged['year_xlsx'].isna() & merged['year_ods'].isna())
                    )
                    prev_year_diff = ~(
                        (merged['previous_year_xlsx'] == merged['previous_year_ods']) |
                        (merged['previous_year_xlsx'].isna() & merged['previous_year_ods'].isna())
                    )
                    
                    differences_found = year_diff.any() or prev_year_diff.any()
                    
                    if differences_found:
                        print(f"    *** DIFFERENCES FOUND ***")
                        print(f"        Year differences: {year_diff.sum()} rows")
                        print(f"        Previous year differences: {prev_year_diff.sum()} rows")
                        count_files_with_error += 1
                        
                        # Store detailed results
                        result = {
                            'subfolder': subfolder.name,
                            'xlsx_file': xlsx_file.name,
                            'ods_file': ods_file.name,
                            'total_merged_rows': len(merged),
                            'total_rows_non_nan': merged.notna().sum().sum(),
                            'year_differences': year_diff.sum(),
                            'prev_year_differences': prev_year_diff.sum(),
                            'differences_data': merged[year_diff | prev_year_diff][[#'E1', 'E2', 'E3', 
                                                                                    'year_xlsx', 'year_ods',
                                                                                    'previous_year_xlsx', 'previous_year_ods']]
                        }
                        results.append(result)
                        
                        # Print some example differences
                        # if len(result['differences_data']) > 0:
                        #     print("        Sample differences:")
                        #     print(result['differences_data'].head())
                    else:
                        print(f"    No differences found in year columns")
                        count_files_without_error += 1
                        
                except Exception as e:
                    print(f"    Error processing files: {str(e)}")
            else:
                print(f"    No corresponding ods file found: {ods_file.name}")
                no_ods.append(xlsx_file.name)
    
    print(f"{count_files_without_error} files without errors")
    print(f"{count_files_with_error} files with errors")
    print(f"Total files processed: {count_files}")
    print(f"no_ods: {no_ods}")

    return results

# Run the analysis
base_path = "/home/simon/Documents/data_science/Thesis/micha_gold_truth_aktiva_only_KORRIGIERT/"
results = check_gold_truth_mistakes(base_path)

Processing subfolder: DZHW
  Found xlsx file: 2023.xlsx
    Found corresponding ods file: aktiva_2023_MYLA.ods
    Loaded xlsx: (29, 5), ods: (29, 5)
    No differences found in year columns
Processing subfolder: Berliner Werstätten für Menschen mit Behinderung
  Found xlsx file: Endgultiger Bericht JAP 2022 BWB.xlsx
    Found corresponding ods file: aktiva_Endgultiger Bericht JAP 2022 BWB_MYLA.ods
    Loaded xlsx: (29, 5), ods: (29, 5)
    *** DIFFERENCES FOUND ***
        Year differences: 0 rows
        Previous year differences: 1 rows
Processing subfolder: Friedrichstadt Palast
  Found xlsx file: 2021-Friedrichstadt-Palast_Jahresbericht.xlsx
    Found corresponding ods file: aktiva_2021-Friedrichstadt-Palast_Jahresbericht_MYLA.ods
    Loaded xlsx: (30, 5), ods: (29, 5)
    *** DIFFERENCES FOUND ***
        Year differences: 1 rows
        Previous year differences: 0 rows
Processing subfolder: Gewobag
  Found xlsx file: Gewobag-Jahresbericht-2021-HGB.xlsx
    Found corresponding o

In [11]:
# Summary
print(f"\n=== SUMMARY ===")
print(f"Total files with differences: {len(results)}")
pd.DataFrame(results)[['subfolder', 'xlsx_file', 'year_differences', 'prev_year_differences']]
# for result in results:
#     print(f"{result['subfolder']}/{result['xlsx_file']}: {result['year_differences']} year diffs, {result['prev_year_differences']} prev_year diffs")


=== SUMMARY ===
Total files with differences: 24


,subfolder,xlsx_file,year_differences,prev_year_differences
0,Berliner Werstätten für Menschen mit Behinderung,Endgultiger Bericht JAP 2022 BWB.xlsx,0,1
1,Friedrichstadt Palast,2021-Friedrichstadt-Palast_Jahresbericht.xlsx,1,0
2,Gewobag,Gewobag-Jahresbericht-2021-HGB.xlsx,3,3
3,WBM GmbH,WBM_GB2020_web_bf.xlsx,1,1
4,ITDZ,jahresbericht-2022.xlsx,4,3
5,Berliner Bäder Betriebe,2018_Geschaeftsbericht_BBB.xlsx,2,2
6,Campus Berlin-Buch,CBB_2021_Pruefungsbericht_digitale_Ausfertigun...,1,1
7,Helmholtz-Zentrum Berlin GmbH,jahresabschluss-2023.xlsx,1,2
8,HOWOGE,HOWOGE_HGB_Konzernabschluss_2023.xlsx,1,1
9,Berliner Wasserbetriebe,2020_geschaeftsbericht-berliner-wasserbetriebe...,1,1


In [3]:
for result in results:
    print(result['differences_data'])

      year_xlsx     year_ods previous_year_xlsx  previous_year_ods
25  20186852.78  20186852.78        16365137,11        16365137.11
    year_xlsx  year_ods  previous_year_xlsx  previous_year_ods
22  343987.25  34987.25           279661.94          279661.94
      year_xlsx     year_ods  previous_year_xlsx  previous_year_ods
5    12663753.0   12663753.0           9423157.0          9423175.0
14          NaN     318844.0                 NaN           323377.0
16     318844.0          NaN            323377.0                NaN
24  396487345.0  396487315.0         232416058.0        232416058.0
    year_xlsx  year_ods  previous_year_xlsx  previous_year_ods
18  2038000.0       NaN           1746000.0                NaN
       year_xlsx     year_ods  previous_year_xlsx  previous_year_ods
5   4.794650e+07  47943498.52        4.313806e+07        43138058.43
18           NaN         0.00                 NaN          327703.73
21  9.667392e+06          NaN        2.130106e+07                Na

In [4]:
import re
import numpy as np

def analyze_difference(row):
    """
    Analyze the difference between xlsx and ods values for possible causes:
    - Comma instead of dot
    - Missing number
    - Swapped number
    """
    results = {}
    for col in ['year', 'previous_year']:
        xlsx_val = row.get(f"{col}_xlsx")
        ods_val = row.get(f"{col}_ods")
        
        # Both NaN: skip
        if pd.isna(xlsx_val) and pd.isna(ods_val):
            results[col] = "both NaN"
            continue
        # One NaN: missing value
        if pd.isna(xlsx_val) or pd.isna(ods_val):
            results[col] = "missing value"
            continue
        
        # Convert to string for analysis
        xlsx_str = str(xlsx_val)
        ods_str = str(ods_val)
        
        # Check for comma/dot confusion
        if ',' in xlsx_str or ',' in ods_str:
            # Try replacing comma with dot and compare as float
            try:
                xlsx_float = float(xlsx_str.replace(',', '.'))
                ods_float = float(ods_str.replace(',', '.'))
                if np.isclose(xlsx_float, ods_float):
                    results[col] = "comma instead of dot"
                    continue
            except Exception:
                pass
        
        # Check for swapped digits (simple case: same length, one swap)
        if len(xlsx_str) == len(ods_str):
            diffs = [(a, b) for a, b in zip(xlsx_str, ods_str) if a != b]
            if len(diffs) == 2 and diffs[0][0] == diffs[1][1] and diffs[0][1] == diffs[1][0]:
                results[col] = "swapped digits"
                continue
        
        # Check for missing digit (length difference of 1)
        if abs(len(xlsx_str) - len(ods_str)) == 1:
            results[col] = "missing digit"
            continue

        if xlsx_str == ods_str:
            results[col] = "same value"
            continue
        
        # Otherwise, just different
        results[col] = "different value"
    return results

# Example: analyze all differences in results
for result in results:
    print(f"\n{result['subfolder']} / {result['xlsx_file']}")
    df = result['differences_data']
    if df.empty:
        print("  No differences data.")
        continue
    for idx, row in df.iterrows():
        analysis = analyze_difference(row)
        print(f"  Row {idx}: {analysis}")


Berliner Werstätten für Menschen mit Behinderung / Endgultiger Bericht JAP 2022 BWB.xlsx
  Row 25: {'year': 'same value', 'previous_year': 'comma instead of dot'}

Friedrichstadt Palast / 2021-Friedrichstadt-Palast_Jahresbericht.xlsx
  Row 22: {'year': 'missing digit', 'previous_year': 'same value'}

Gewobag / Gewobag-Jahresbericht-2021-HGB.xlsx
  Row 5: {'year': 'same value', 'previous_year': 'swapped digits'}
  Row 14: {'year': 'missing value', 'previous_year': 'missing value'}
  Row 16: {'year': 'missing value', 'previous_year': 'missing value'}
  Row 24: {'year': 'different value', 'previous_year': 'same value'}

WBM GmbH / WBM_GB2020_web_bf.xlsx
  Row 18: {'year': 'missing value', 'previous_year': 'missing value'}

ITDZ / jahresbericht-2022.xlsx
  Row 5: {'year': 'different value', 'previous_year': 'same value'}
  Row 18: {'year': 'missing value', 'previous_year': 'missing value'}
  Row 21: {'year': 'missing value', 'previous_year': 'missing value'}
  Row 25: {'year': 'missing di

In [6]:
from collections import Counter

error_counter = Counter()

for result in results:
    df = result['differences_data']
    if df.empty:
        continue
    for _, row in df.iterrows():
        analysis = analyze_difference(row)
        for err in analysis.values():
            error_counter[err] += 1

print("Error type counts:")
for err_type, count in error_counter.items():
    print(f"{err_type}: {count}")

Error type counts:
same value: 21
comma instead of dot: 1
missing digit: 10
swapped digits: 2
missing value: 49
different value: 13


In [18]:
total_non_nan_ods = sum(result.get('total_rows_non_nan', 0) for result in results)
print(f"Total non-NaN values in all used ods files: {total_non_nan_ods}")


Total non-NaN values in all used ods files: 3088


In [19]:
total_errors = sum(count for err, count in error_counter.items() if err != 'same value')
print(f"Total errors (excluding 'same value'): {total_errors}")

Total errors (excluding 'same value'): 75


In [20]:
print(round(total_errors/total_non_nan_ods*100, 1))

2.4
